# Proje Üyeleri:
1) Umut Özdemir         211101004
2) Kadir Bora Kara      201101029
3) Can Ata Haccacoglu   201101033

# Script Kullanımı Açıklama:
Script'imiz bir Networkx graph'ı alıyor ve bu graph'ı 30 dimension'lı bir graph embedding'ine çeviriyor
Bu graph embedding'ini Node2Vec kütüphanesini kullanarak elde etmekteyiz 
Parameters:
1) walk_length = 10 
2) num_walks = 40

pickle kütüphanesini kullanarak elde ettiğimiz eğitilmiş kMeans için gerekli olan modelleri load ediyoruz.
Load Ettiğimiz modellerle bize verilen graph'ın sonucunu predict ederek bu predict edilen sonucu dönüyoruz.

# Kullanma yöntemi:
Networkx graph'ınızı "scriptKullanilacakFonksiyon" isimli fonksiyona vermeniz yeterlidir.
Bu fonksiyon graph'ınızı alacak ve sonucunda prediction'ı dönecektir.
Fonksiyonda dönen değer 1 ise Partition edilebilir (Yani Yes label'lı), 0 ise Partition edilemez(Yani No label'li)

# Gerekli kutuphaneler
1) numpy
2) node2Vec (Version 0.4.6)
3) pickle   (Version 0.7.5)

In [1]:
# General Libraries
import random as rand
import numpy as np
# Graph Representation & Embedding Library
import networkx as nx 
from node2vec import Node2Vec
import pickle

c:\Users\bora\anaconda3\envs\test1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def getEmbedding(G):
    # Precompute probabilities and generate walks
    node2vec = Node2Vec(G, dimensions=30, walk_length=10, num_walks=40, workers=4)

    # Embed nodes
    model = node2vec.fit(window=5, min_count=1, batch_words=4)

    # Get embeddings for all nodes in the graph
    all_node_embeddings = {node: model.wv[str(node)] for node in G.nodes()}

    return all_node_embeddings
    # loaded_model = Node2Vec.load("node2vec_model.bin")
    # loaded_embedding = loaded_model.wv['0']
    # print(f"Loaded Embedding for Node 0: {loaded_embedding}")

def dictionaryToNpArray(embedding):
    array_from_dict = np.array(list(embedding.values()))
    return array_from_dict


In [3]:
def scriptKullanilacakFonksiyon(G):
    # Modeli yükleyip kullanma
    # K-Means modelini yükle
    with open('logistic_regression_model.pickle', 'rb') as model_file:
        loaded_classifier = pickle.load(model_file)

    nodeEmbeddings = getEmbedding(G) 
    nodeEmbeddingsArray = dictionaryToNpArray(nodeEmbeddings)
    graphEmbedding = np.mean(nodeEmbeddingsArray, axis=0).astype('float')

    result_prediction = loaded_classifier.predict(graphEmbedding.reshape(1, -1))

    # Prediction 0.5'den buyukse 1 veriyor (Yani partition edilebilir)
    if result_prediction > 0.5:
        return 1
    
    # Prediction 0.5'den kucukse 0 veriyor (Yani partition edilemedi)
    else: 
        return 0
    


In [4]:
G = nx.connected_watts_strogatz_graph(n=10, k=3, p=0.3)
print(scriptKullanilacakFonksiyon(G))

c:\Users\bora\anaconda3\envs\test1\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
Computing transition probabilities: 100%|██████████| 10/10 [00:00<00:00, 10007.88it/s]


1
